# Inverse Laplace transform evaluation of chain diagrams

In [3]:
using InverseLaplace: Talbot
using Printf

"""
Make the Laplace image of d_n(α, x) w.r.t. x
"""
function make_Ldn(n, α)
    return s -> factorial(big(2n)) / (s^2 * (s + α)^n * (s + 2α)^(n-1))
end

# Multi-precision Laplace transform inversion
# Abate, J. and Valkó, P.P.
# International Journal for Numerical Methods in Engineering, Vol. 60 (Iss. 5-7)  2004  pp 979–993
# https://doi.org/10.1002/nme.995

setprecision(BigFloat, 4000)

for α in [0.0, 0.5, 1.0, 2.0]
    for n in [1, 2, 3, 4, 5, 10, 1000, 2000, 4000]
        Nterms = max(2n, 100) # max() ensures we use enough terms at low n
        d_n = Talbot(make_Ldn(n, α), Nterms)(big(1.0))
        @printf "α = %.3f, n = %5i\t d_n = %.10e\n" α n d_n;
    end
end

α = 0.000, n =     1	 d_n = 1.0000000000e+00
α = 0.000, n =     2	 d_n = 1.0000000000e+00
α = 0.000, n =     3	 d_n = 1.0000000000e+00
α = 0.000, n =     4	 d_n = 1.0000000000e+00
α = 0.000, n =     5	 d_n = 1.0000000000e+00
α = 0.000, n =    10	 d_n = 1.0000000000e+00
α = 0.000, n =  1000	 d_n = 1.0000000000e+00
α = 0.000, n =  2000	 d_n = 1.0000000000e+00
α = 0.000, n =  4000	 d_n = 1.0000000000e+00
α = 0.500, n =     1	 d_n = 8.5224527770e-01
α = 0.500, n =     2	 d_n = 6.7808634694e-01
α = 0.500, n =     3	 d_n = 6.1196873349e-01
α = 0.500, n =     4	 d_n = 5.7767763926e-01
α = 0.500, n =     5	 d_n = 5.5680280079e-01
α = 0.500, n =    10	 d_n = 5.1462739416e-01
α = 0.500, n =  1000	 d_n = 4.7278728641e-01
α = 0.500, n =  2000	 d_n = 4.7257691154e-01
α = 0.500, n =  4000	 d_n = 4.7247173013e-01
α = 1.000, n =     1	 d_n = 7.3575888234e-01
α = 1.000, n =     2	 d_n = 4.7022487565e-01
α = 1.000, n =     3	 d_n = 3.8123919564e-01
α = 1.000, n =     4	 d_n = 3.3832491320e-01
α = 1.000,